In [2]:
import ipyvuetify as v

In [3]:
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from bqplot import pyplot as plt, LinearScale, Axis, Bars, OrdinalScale , Label
import requests
import bqplot
from datetime import datetime


# First histogram plot

## Profits by warehouses


In [4]:
# Get warehouse IDs
warehouse_ids_url = "https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/"
response = requests.get(warehouse_ids_url)
warehouse_data = response.json()

#clean nulls
for warehouse in warehouse_data:
    if not warehouse.get('Budget') or warehouse['Budget'] is None:
        warehouse['Budget'] = '0'

#gather ids for next post
warehouse_ids_profits = sorted([warehouse['WarehouseID'] for warehouse in warehouse_data])
warehouse_id_options_profits = [{'text': str(id), 'value': id} for id in warehouse_ids_profits]

# Create scales and bar chart (initially empty)
x_sc = OrdinalScale()
y_sc = LinearScale()
bar_chart_3 = Bars(x=[], y=[], scales={'x': x_sc, 'y': y_sc})
# Creating axes with labels
x_ax = Axis(label='Year', scale=x_sc)  # Label for x-axis
y_ax = Axis(label='Profit  ', scale=y_sc, orientation='vertical', tick_format='0.2f')  # Label for y-axis

bar_labels_3 = Label(x=[], y=[], text=[], scales={'x': x_sc, 'y': y_sc}, 
                   colors=['black'], default_size=15, 
                   update_on_move=True, align='middle')

fig3 = plt.Figure(marks=[bar_chart_3, bar_labels_3], axes=[x_ax, y_ax], title='Profits for Selected Warehouse')
fig3.layout.width = 'auto'
fig3.layout.height = 'auto'
fig3.layout.min_height = '300px'


def fetch_profit_data(warehouse_id):
    profit_url = f"https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/{warehouse_id}/profit"
    response = requests.post(profit_url, json={"UserID": 2})
    profit_data = response.json()
    return (profit_data)


def on_dropdown_change_profit(change):
    if change['type'] == 'change' and change['name'] == 'v_model':
        warehouse_id = change['new']
        new_profit_data = fetch_profit_data(warehouse_id)

        # Check if the response is a message indicating no records
        if isinstance(new_profit_data, dict) and "message" in new_profit_data:
            # Update the figure with zero values and the message
            bar_chart_3.set_trait('x', [])
            bar_chart_3.set_trait('y', [])
            bar_labels_3.set_trait('x', [])
            bar_labels_3.set_trait('y', [])
            bar_labels_3.set_trait('text', [])
            fig3.title = new_profit_data["message"]
        else:
            # Proceed as normal if data is present
            years = [d['Year'] for d in new_profit_data]
            profits = [d['Profit'] for d in new_profit_data]

            # Update bar_chart and bar_labels with actual data
            bar_chart_3.set_trait('x', years)
            bar_chart_3.set_trait('y', profits)
            adjusted_profits = [profit + max(profits) * 0.05 for profit in profits]
            bar_labels_3.set_trait('x', years)
            bar_labels_3.set_trait('y', adjusted_profits)
            bar_labels_3.set_trait('text', [f'{profit}' for profit in profits])

            # Updating the title
            fig3.title = f'Profits for Warehouse ID {warehouse_id}'


# Create a vuetify dropdown widget for warehouse IDs
dropdown = v.Select(items=warehouse_id_options_profits, v_model=warehouse_ids_profits[0], label="Select Warehouse ID")

# Setting the callback for the dropdown
dropdown.observe(on_dropdown_change_profit)

display(fig3)
display(dropdown)

# Initial data load
on_dropdown_change_profit({'new': warehouse_ids_profits[0], 'type': 'change', 'name': 'v_model'})


Figure(axes=[Axis(label='Year', scale=OrdinalScale()), Axis(label='Profit  ', orientation='vertical', scale=Li…

Select(items=[{'text': '3', 'value': 3}, {'text': '4', 'value': 4}, {'text': '5', 'value': 5}, {'text': '6', '…

## top 5 racks with quantity under de 25 % capacity threshold


In [11]:
# Get warehouse IDs
warehouse_ids_url = "https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/"
response = requests.get(warehouse_ids_url)
warehouse_data = response.json()

#clean nulls
for warehouse in warehouse_data:
    if not warehouse.get('Budget') or warehouse['Budget'] is None:
        warehouse['Budget'] = '0'

#gather ids for next post
warehouse_ids = sorted([warehouse['WarehouseID'] for warehouse in warehouse_data])
warehouse_id_options = [{'text': str(id), 'value': id} for id in warehouse_ids]

# Create scales and bar chart (initially empty)
x_sc = OrdinalScale()
y_sc = LinearScale()

bar_chart_4 = Bars(x=[], y=[], scales={'x': x_sc, 'y': y_sc})
# Creating axes with labels
x_ax = Axis(label='Rack ID', scale=x_sc)  # Label for x-axis
y_ax = Axis(label='Rack Capacity %', scale=y_sc, orientation='vertical', tick_format='0.2f')  # Label for y-axis

bar_labels_4 = Label(x=[], y=[], text=[], scales={'x': x_sc, 'y': y_sc}, 
                   colors=['black'], default_size=15, 
                   update_on_move=True, align='middle')

fig4 = plt.Figure(marks=[bar_chart_4, bar_labels_4], axes=[x_ax, y_ax], title="Top 5 racks with quantity under the 25% capacity threshold for Warehouse")
fig4.layout.width = 'auto'
fig4.layout.height = 'auto'
fig4.layout.min_height = '300px'


def fetch_rack_data(warehouse_id):
    profit_url = f"https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/{warehouse_id}/rack/lowstock"
    response = requests.post(profit_url, json={"UserID": 2})
    profit_data = response.json()
    return (profit_data)

# Function to update the graph based on dropdown selection
def on_dropdown_change_rack(change):
    if change['type'] == 'change' and change['name'] == 'v_model':
        warehouse_id = change['new']
        new_rack_data = fetch_rack_data(warehouse_id) 
        # Check if the response is a message indicating no records
        if isinstance(new_rack_data, dict) and "message" in new_rack_data:
            # Update the figure with zero values and the message
            bar_chart_4.set_trait('x', [])
            bar_chart_4.set_trait('y', [])
            bar_labels_4.set_trait('x', [])
            bar_labels_4.set_trait('y', [])
            bar_labels_4.set_trait('text', [])
            fig4.title = new_rack_data["message"]
        else:
            # Extracting RackID and Rack Capacity
            rack_ids = [str(rack['RackID']) for rack in new_rack_data]
            rack_capacities = [float(rack['Rack Capacity'].rstrip('%')) for rack in new_rack_data]

            bar_chart_4.x = rack_ids
            bar_chart_4.y = rack_capacities

            # Update labels
            bar_labels_4.x = rack_ids
            bar_labels_4.y = [capacity + max(rack_capacities) * 0.05 for capacity in rack_capacities]  # Slightly above each bar
            bar_labels_4.text = [f'{capacity}%' for capacity in rack_capacities]

            fig4.title = f'Rack Capacities for Warehouse ID {warehouse_id}'



# Create a vuetify dropdown widget for warehouse IDs
dropdown4 = v.Select(items=warehouse_id_options, v_model=warehouse_ids[0], label="Select Warehouse ID")

# Setting the callback for the dropdown
dropdown4.observe(on_dropdown_change_rack)

display(fig4)
display(dropdown4)

# Initial data load
on_dropdown_change_rack({'new': warehouse_ids[0], 'type': 'change', 'name': 'v_model'})


Figure(axes=[Axis(label='Rack ID', scale=OrdinalScale()), Axis(label='Rack Capacity %', orientation='vertical'…

Select(items=[{'text': '3', 'value': 3}, {'text': '4', 'value': 4}, {'text': '5', 'value': 5}, {'text': '6', '…

## tab 5

In [20]:
# Get warehouse IDs
warehouse_ids_url = "https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/"
response = requests.get(warehouse_ids_url)
warehouse_data = response.json()

#clean nulls
for warehouse in warehouse_data:
    if not warehouse.get('Budget') or warehouse['Budget'] is None:
        warehouse['Budget'] = '0'

#gather ids for next post
warehouse_ids_5 = sorted([warehouse['WarehouseID'] for warehouse in warehouse_data])
warehouse_id_options_5 = [{'text': str(id), 'value': id} for id in warehouse_ids_5]

# Create scales and bar chart (initially empty)
x_sc = OrdinalScale()
y_sc = LinearScale()
bar_chart_5 = Bars(x=[], y=[], scales={'x': x_sc, 'y': y_sc})
# Creating axes with labels
x_ax = Axis(label='Material', scale=x_sc)  # Label for x-axis
y_ax = Axis(label='Part Count  ', scale=y_sc, orientation='vertical', tick_format='0.2f')  # Label for y-axis

bar_labels_5 = Label(x=[], y=[], text=[], scales={'x': x_sc, 'y': y_sc}, 
                   colors=['black'], default_size=15, 
                   update_on_move=True, align='middle')

fig5 = plt.Figure(marks=[bar_chart_5, bar_labels_5], axes=[x_ax, y_ax], title='Profits for Selected Warehouse')
fig5.layout.width = 'auto'
fig5.layout.height = 'auto'
fig5.layout.min_height = '300px'


def fetch_data_5(warehouse_id):
    profit_url = f"https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/{warehouse_id}/rack/material"
    response = requests.post(profit_url, json={"UserID": 2})
    profit_data = response.json()
    return (profit_data)


def on_dropdown_change_5(change):
    if change['type'] == 'change' and change['name'] == 'v_model':
        warehouse_id = change['new']
        new_data_5 = fetch_data_5(warehouse_id)

        # Check if the response is a message indicating no records
        if isinstance(new_data_5, dict) and "message" in new_data_5:
            bar_chart_5.set_trait('x', [])
            bar_chart_5.set_trait('y', [])
            bar_labels_5.set_trait('x', [])
            bar_labels_5.set_trait('y', [])
            bar_labels_5.set_trait('text', [])
            fig5.title = new_data_5["message"]
        else:
            # Map the Part Material numbers to meaningful labels
            material_labels = {1: "Metal", 2: "Rubber", 3: "Polymer", 4: "Fluid"}
            materials = [material_labels[d["Part Material 1=Metal, 2=Rubber, 3=Polymer, 4=Fluid"]] for d in new_data_5]
            counts = [d["Part Count in Warehouse"] for d in new_data_5]

            bar_chart_5.set_trait('x', materials)
            bar_chart_5.set_trait('y', counts)

            # Update labels
            bar_labels_5.set_trait('x', materials)
            bar_labels_5.set_trait('y', [count + max(counts) * 0.05 for count in counts])
            bar_labels_5.set_trait('text', [str(count) for count in counts])

            fig5.title = f'Bottom 3 Part Counts for Warehouse ID {warehouse_id}'



# Create a vuetify dropdown widget for warehouse IDs
dropdown5 = v.Select(items=warehouse_id_options_5, v_model=warehouse_ids_5[0], label="Select Warehouse ID")

# Setting the callback for the dropdown
dropdown5.observe(on_dropdown_change_5)

display(fig5)
display(dropdown5)

# Initial data load
on_dropdown_change_5({'new': warehouse_ids_5[0], 'type': 'change', 'name': 'v_model'})


Figure(axes=[Axis(label='Material', scale=OrdinalScale()), Axis(label='Part Count  ', orientation='vertical', …

Select(items=[{'text': '3', 'value': 3}, {'text': '4', 'value': 4}, {'text': '5', 'value': 5}, {'text': '6', '…

## tab 6

In [21]:
# Get warehouse IDs
warehouse_ids_url = "https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/"
response = requests.get(warehouse_ids_url)
warehouse_data = response.json()

#clean nulls
for warehouse in warehouse_data:
    if not warehouse.get('Budget') or warehouse['Budget'] is None:
        warehouse['Budget'] = '0'

#gather ids for next post
warehouse_ids_6 = sorted([warehouse['WarehouseID'] for warehouse in warehouse_data])
warehouse_id_options_6 = [{'text': str(id), 'value': id} for id in warehouse_ids_6]

# Create scales and bar chart (initially empty)
x_sc = OrdinalScale()
y_sc = LinearScale()
bar_chart_6 = Bars(x=[], y=[], scales={'x': x_sc, 'y': y_sc})
# Creating axes with labels
x_ax = Axis(label='Rack ID', scale=x_sc)  # Label for x-axis
y_ax = Axis(label='Rack Total Value  ', scale=y_sc, orientation='vertical', tick_format='0.2f')  # Label for y-axis

bar_labels_6 = Label(x=[], y=[], text=[], scales={'x': x_sc, 'y': y_sc}, 
                   colors=['black'], default_size=13, 
                   update_on_move=True, align='middle')

fig6 = plt.Figure(marks=[bar_chart_6, bar_labels_6], axes=[x_ax, y_ax], title='Profits for Selected Warehouse')
fig6.layout.width = 'auto'
fig6.layout.height = 'auto'
fig6.layout.min_height = '300px'


def fetch_data_6(warehouse_id):
    profit_url = f"https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/{warehouse_id}/rack/expensive"
    response = requests.post(profit_url, json={"UserID": 2})
    profit_data = response.json()
    return (profit_data)


def on_dropdown_change_6(change):
    if change['type'] == 'change' and change['name'] == 'v_model':
        warehouse_id = change['new']
        new_data_6 = fetch_data_6(warehouse_id)

        # Check if the response is a message indicating no records
        if isinstance(new_data_6, dict) and "message" in new_data_6:
            bar_chart_6.set_trait('x', [])
            bar_chart_6.set_trait('y', [])
            bar_labels_6.set_trait('x', [])
            bar_labels_6.set_trait('y', [])
            bar_labels_6.set_trait('text', [])
            fig6.title = new_data_6["message"]
        else:
            # Extracting RackID and Rack Total Value
            rack_ids = [str(rack['RackID']) for rack in new_data_6]
            rack_values = [rack['Rack Total Value'] for rack in new_data_6]

            bar_chart_6.set_trait('x', rack_ids)
            bar_chart_6.set_trait('y', rack_values)

            # Update labels
            adjusted_values = [value + max(rack_values) * 0.05 for value in rack_values]  # Slightly above each bar
            bar_labels_6.set_trait('x', rack_ids)
            bar_labels_6.set_trait('y', adjusted_values)
            bar_labels_6.set_trait('text', [f'{value}' for value in rack_values])

            fig6.title = f'Top 5 expensive racks for Warehouse ID {warehouse_id}'



# Create a vuetify dropdown widget for warehouse IDs
dropdown6 = v.Select(items=warehouse_id_options_6, v_model=warehouse_ids_6[0], label="Select Warehouse ID")

# Setting the callback for the dropdown
dropdown6.observe(on_dropdown_change_6)

display(fig6)
display(dropdown6)

# Initial data load
on_dropdown_change_6({'new': warehouse_ids_6[0], 'type': 'change', 'name': 'v_model'})


Figure(axes=[Axis(label='Rack ID', scale=OrdinalScale()), Axis(label='Rack Total Value  ', orientation='vertic…

Select(items=[{'text': '3', 'value': 3}, {'text': '4', 'value': 4}, {'text': '5', 'value': 5}, {'text': '6', '…

## tab7

In [24]:
# Get warehouse IDs
warehouse_ids_url = "https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/"
response = requests.get(warehouse_ids_url)
warehouse_data = response.json()

#clean nulls
for warehouse in warehouse_data:
    if not warehouse.get('Budget') or warehouse['Budget'] is None:
        warehouse['Budget'] = '0'

#gather ids for next post
warehouse_ids_7 = sorted([warehouse['WarehouseID'] for warehouse in warehouse_data])
warehouse_id_options_7 = [{'text': str(id), 'value': id} for id in warehouse_ids_7]

# Create scales and bar chart (initially empty)
x_sc = OrdinalScale()
y_sc = LinearScale()
bar_chart_7 = Bars(x=[], y=[], scales={'x': x_sc, 'y': y_sc})
# Creating axes with labels
x_ax = Axis(label='Date', scale=x_sc)  # Label for x-axis
y_ax = Axis(label='Profit  ', scale=y_sc, orientation='vertical', tick_format='0.2f')  # Label for y-axis

bar_labels_7 = Label(x=[], y=[], text=[], scales={'x': x_sc, 'y': y_sc}, 
                   colors=['black'], default_size=15, 
                   update_on_move=True, align='middle')

fig7 = plt.Figure(marks=[bar_chart_7, bar_labels_7], axes=[x_ax, y_ax], title='Incoming transactions for Selected Warehouse')
fig7.layout.width = 'auto'
fig7.layout.height = 'auto'
fig7.layout.min_height = '300px'


def fetch_data_7(warehouse_id):
    profit_url = f"https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/{warehouse_id}/transaction/leastcost"
    response = requests.post(profit_url, json={"UserID": 2})
    profit_data = response.json()
    return (profit_data)


def on_dropdown_change_7(change):
    if change['type'] == 'change' and change['name'] == 'v_model':
        warehouse_id = change['new']
        new_data_7 = fetch_data_7(warehouse_id)  # This function should fetch the new data format

        # Check if the response is a message indicating no records
        if isinstance(new_data_7, dict) and "message" in new_data_7:
            bar_chart_7.set_trait('x', [])
            bar_chart_7.set_trait('y', [])
            bar_labels_7.set_trait('x', [])
            bar_labels_7.set_trait('y', [])
            bar_labels_7.set_trait('text', [])
            fig7.title = new_data_7["message"]
        else:
            # Extracting data from the new format
            transaction_dates = [datetime.strptime(d["TransactionDate"], "%a, %d %b %Y %H:%M:%S %Z").strftime("%Y-%m-%d") for d in new_data_7]
            profits = [d["Profit"] for d in new_data_7]

            bar_chart_7.set_trait('x', transaction_dates)
            bar_chart_7.set_trait('y', profits)

            # Update labels
            adjusted_profits = [profit + max(profits) * 0.05 for profit in profits]
            bar_labels_7.set_trait('x', transaction_dates)
            bar_labels_7.set_trait('y', adjusted_profits)
            bar_labels_7.set_trait('text', [f'{profit}' for profit in profits])

            fig7.title = f'Bottom 3 incoming transaction days for Warehouse ID {warehouse_id}'

# Create a vuetify dropdown widget for warehouse IDs
dropdown7 = v.Select(items=warehouse_id_options_7, v_model=warehouse_ids_7[0], label="Select Warehouse ID")

# Setting the callback for the dropdown
dropdown7.observe(on_dropdown_change_7)

display(fig7)
display(dropdown7)

# Initial data load
on_dropdown_change_7({'new': warehouse_ids_7[0], 'type': 'change', 'name': 'v_model'})


Figure(axes=[Axis(label='Date', scale=OrdinalScale()), Axis(label='Profit  ', orientation='vertical', scale=Li…

Select(items=[{'text': '3', 'value': 3}, {'text': '4', 'value': 4}, {'text': '5', 'value': 5}, {'text': '6', '…

## tab 8

In [15]:
# Get warehouse IDs
warehouse_ids_url = "https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/"
response = requests.get(warehouse_ids_url)
warehouse_data = response.json()

#clean nulls
for warehouse in warehouse_data:
    if not warehouse.get('Budget') or warehouse['Budget'] is None:
        warehouse['Budget'] = '0'

#gather ids for next post
warehouse_ids_8 = sorted([warehouse['WarehouseID'] for warehouse in warehouse_data])
warehouse_id_options_8 = [{'text': str(id), 'value': id} for id in warehouse_ids_8]

# Create scales and bar chart (initially empty)
x_sc = OrdinalScale()
y_sc = LinearScale()
bar_chart_8 = Bars(x=[], y=[], scales={'x': x_sc, 'y': y_sc})
# Creating axes with labels
x_ax = Axis(label='Supplier', scale=x_sc)  # Label for x-axis
y_ax = Axis(label='Supplies Count  ', scale=y_sc, orientation='vertical', tick_format='0.2f')  # Label for y-axis

bar_labels_8 = Label(x=[], y=[], text=[], scales={'x': x_sc, 'y': y_sc}, 
                   colors=['black'], default_size=15, 
                   update_on_move=True, align='middle')

fig8 = plt.Figure(marks=[bar_chart_8, bar_labels_8], axes=[x_ax, y_ax], title='Profits for Selected Warehouse')
fig8.layout.width = 'auto'
fig8.layout.height = 'auto'
fig8.layout.min_height = '300px'


def fetch_data_8(warehouse_id):
    profit_url = f"https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/{warehouse_id}/transaction/suppliers"
    response = requests.post(profit_url, json={"UserID": 2})
    profit_data = response.json()
    return (profit_data)



def on_dropdown_change_8(change):
    if change['type'] == 'change' and change['name'] == 'v_model':
        warehouse_id = change['new']
        new_data_8 = fetch_data_8(warehouse_id)

        # Check if the response is a message indicating no records
        if isinstance(new_data_8, dict) and "message" in new_data_8:
            bar_chart_8.set_trait('x', [])
            bar_chart_8.set_trait('y', [])
            bar_labels_8.set_trait('x', [])
            bar_labels_8.set_trait('y', [])
            bar_labels_8.set_trait('text', [])
            fig8.title = new_data_8["message"]
        else:
            supplier_ids = [f"Supplier {d['SupplierID']}" for d in new_data_8]
            supply_counts = [d[f"Count of Supplies Supplied to Warehouse #{warehouse_id}"] for d in new_data_8]

            bar_chart_8.set_trait('x', supplier_ids)
            bar_chart_8.set_trait('y', supply_counts)

            # Update labels
            bar_labels_8.set_trait('x', supplier_ids)
            bar_labels_8.set_trait('y', [count + max(supply_counts) * 0.05 for count in supply_counts])
            bar_labels_8.set_trait('text', [str(count) for count in supply_counts])

            fig8.title = f'Most exchanges for Warehouse ID {warehouse_id}'


# Create a vuetify dropdown widget for warehouse IDs
dropdown8 = v.Select(items=warehouse_id_options_8, v_model=warehouse_ids_8[0], label="Select Warehouse ID")

# Setting the callback for the dropdown
dropdown8.observe(on_dropdown_change_8)

display(fig8)
display(dropdown8)

# Initial data load
on_dropdown_change_8({'new': warehouse_ids_8[0], 'type': 'change', 'name': 'v_model'})


Figure(axes=[Axis(label='Supplier', scale=OrdinalScale()), Axis(label='Supplies Count  ', orientation='vertica…

Select(items=[{'text': '3', 'value': 3}, {'text': '4', 'value': 4}, {'text': '5', 'value': 5}, {'text': '6', '…

## tab9

In [16]:
# Get warehouse IDs
warehouse_ids_url = "https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/"
response = requests.get(warehouse_ids_url)
warehouse_data = response.json()

#clean nulls
for warehouse in warehouse_data:
    if not warehouse.get('Budget') or warehouse['Budget'] is None:
        warehouse['Budget'] = '0'

#gather ids for next post
warehouse_ids_9 = sorted([warehouse['WarehouseID'] for warehouse in warehouse_data])
warehouse_id_options_9 = [{'text': str(id), 'value': id} for id in warehouse_ids_9]

# Create scales and bar chart (initially empty)
x_sc = OrdinalScale()
y_sc = LinearScale()
bar_chart_9 = Bars(x=[], y=[], scales={'x': x_sc, 'y': y_sc})
# Creating axes with labels
x_ax = Axis(label='User', scale=x_sc)  # Label for x-axis
y_ax = Axis(label='Transactions  ', scale=y_sc, orientation='vertical', tick_format='0.2f')  # Label for y-axis

bar_labels_9 = Label(x=[], y=[], text=[], scales={'x': x_sc, 'y': y_sc}, 
                   colors=['black'], default_size=15, 
                   update_on_move=True, align='middle')

fig9 = plt.Figure(marks=[bar_chart_9, bar_labels_9], axes=[x_ax, y_ax], title='Profits for Selected Warehouse')
fig9.layout.width = 'auto'
fig9.layout.height = 'auto'
fig9.layout.min_height = '300px'


def fetch_data_9(warehouse_id):
    profit_url = f"https://postgres-sql-6aba726b2968.herokuapp.com/jemsa/warehouse/{warehouse_id}/users/receivesmost"
    response = requests.post(profit_url, json={"UserID": 2})
    profit_data = response.json()
    return (profit_data)

def on_dropdown_change_9(change):
    if change['type'] == 'change' and change['name'] == 'v_model':
        warehouse_id = change['new']
        new_data_9 = fetch_data_9(warehouse_id)

        # Check if the response is a message indicating no records
        if isinstance(new_data_9, dict) and "message" in new_data_9:
            bar_chart_9.set_trait('x', [])
            bar_chart_9.set_trait('y', [])
            bar_labels_9.set_trait('x', [])
            bar_labels_9.set_trait('y', [])
            bar_labels_9.set_trait('text', [])
            fig9.title = new_data_9["message"]
        else:
            # Extract usernames and transaction counts from the data
            usernames = [d["Username"] for d in new_data_9]
            transaction_counts = [d["Transaction_Count"] for d in new_data_9]

            bar_chart_9.set_trait('x', usernames)
            bar_chart_9.set_trait('y', transaction_counts)

            # Update labels
            bar_labels_9.set_trait('x', usernames)
            bar_labels_9.set_trait('y', [count + max(transaction_counts) * 0.05 for count in transaction_counts])
            bar_labels_9.set_trait('text', [str(count) for count in transaction_counts])

            fig9.title = f'Transaction Counts for Warehouse ID {warehouse_id}'




# Create a vuetify dropdown widget for warehouse IDs
dropdown9 = v.Select(items=warehouse_id_options_9, v_model=warehouse_ids_9[0], label="Select Warehouse ID")

# Setting the callback for the dropdown
dropdown9.observe(on_dropdown_change_9)

display(fig9)
display(dropdown9)

# Initial data load
on_dropdown_change_9({'new': warehouse_ids_9[0], 'type': 'change', 'name': 'v_model'})


Figure(axes=[Axis(label='User', scale=OrdinalScale()), Axis(label='Transactions  ', orientation='vertical', sc…

Select(items=[{'text': '3', 'value': 3}, {'text': '4', 'value': 4}, {'text': '5', 'value': 5}, {'text': '6', '…

# Set up voila vuetify layout
The voila vuetify template does not render output from the notebook, it only shows widget with the mount_id metadata.

In [17]:
v.Layout(pa_4=True, _metadata={'mount_id': 'content-nav'}, column=True, children=[]);

In [18]:

v.Tabs(_metadata={'mount_id': 'content-main'}, children=[

    v.Tab(children=['Warehouse’s year profit']),
    v.Tab(children=['Top 5 racks with quantity under the 25% capacity  threshold']),
    v.Tab(children=['Bottom 3 part’s type/material in the warehouse']),
    v.Tab(children=['Top 5 most expensive racks in the warehouse']),
    v.Tab(children=['Top 3 supplier that supplied to the warehouse']),
    v.Tab(children=['Top 3 days with the smallest incoming transactions']),
    v.Tab(children=['Top 3 users that receives the most exchanges']),

    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True,justify_center=True, children=[
            # Flex container for the figure
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                fig3
            ]),
            # Flex container for the dropdown
            v.Flex(xs4=True, align_center=True ,lg2=True, xl1=True, children=[
                dropdown
            ])

        ])
    ]),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True,justify_center=True, children=[
            # Flex container for the figure
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                fig4
            ]),
            # Flex container for the dropdown
            v.Flex(xs4=True, align_center=True ,lg2=True, xl1=True, children=[
                dropdown4
            ])

        ])
    ]),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True,justify_center=True, children=[
            # Flex container for the figure
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                fig5
            ]),
            # Flex container for the dropdown
            v.Flex(xs4=True, align_center=True ,lg2=True, xl1=True, children=[
                dropdown5
            ])

        ])
    ]),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True,justify_center=True, children=[
            # Flex container for the figure
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                fig6
            ]),
            # Flex container for the dropdown
            v.Flex(xs4=True, align_center=True ,lg2=True, xl1=True, children=[
                dropdown6
            ])

        ])
    ]),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True,justify_center=True, children=[
            # Flex container for the figure
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                fig8
            ]),
            # Flex container for the dropdown
            v.Flex(xs4=True, align_center=True ,lg2=True, xl1=True, children=[
                dropdown8
            ])

        ])
    ]),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True,justify_center=True, children=[
            # Flex container for the figure
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                fig7
            ]),
            # Flex container for the dropdown
            v.Flex(xs4=True, align_center=True ,lg2=True, xl1=True, children=[
                dropdown7
            ])

        ])
    ]),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True,justify_center=True, children=[
            # Flex container for the figures
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                fig9
            ]),
            # Flex container for the dropdown
            v.Flex(xs4=True, align_center=True ,lg2=True, xl1=True, children=[
                dropdown9
            ])

        ])
    ])
])

Tabs(children=[Tab(children=['Warehouse’s year profit'], layout=None), Tab(children=['Top 5 racks with quantit…